http://localhost:8888/?token=f7038fd1d7ab2a2e2e48f6361fceaf17ef4e15dc392523b0

In [1]:
import json
import time 

from kafka import KafkaProducer

In [3]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'redpanda-1:29092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [4]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

print(time.time()-t0)
producer.flush()
print(time.time()-t0)
t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
0.5415644645690918
0.5420417785644531
took 0.54 seconds


In [ ]:
import pandas as pd

In [ ]:
df_green = pd.read_csv('https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz', compression='gzip')

In [ ]:
df_green  = df_green[['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'tip_amount']]

In [ ]:
df_green.head()

In [ ]:
t0 = time.time()
topic_name = 'green-trips'
for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=row_dict)

producer.flush()
time.time() - t0

In [5]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [4]:
import pkg_resources

# Get the list of installed packages
installed_packages = pkg_resources.working_set

# Print the installed packages
for package in installed_packages:
    if 'spark' in package.key:
        print(package.key)
    # print(f"{package.key}=={package.version}")